In [20]:
logfile = 'logs/cudnn1.log'

blocks = []
with open(logfile, 'r') as logfile:
    while(True):
        line = logfile.readline()
        if not line:
            break
        # do stuff
        if 'cudnnConvolutionForward()' in line:
            block = line
            line = logfile.readline()
            while(line.strip()):
                block += line
                line = logfile.readline()
            blocks.append(block)
print(len(blocks))

104


In [21]:
import re

convsInfo = []
for i, block in enumerate(blocks):
    conv = {}
    
    lines = block.split('\n')
    
    ## INPUT INFORMATION
    xDescLine = [x for x in range(len(lines)) if 'xDesc' in lines[x]]
    assert len(xDescLine) == 1, "multiple 'xDesc' in block:{}".format(i+1)
    xDescLine = xDescLine[0]
    
    xPattern = re.compile(r'i!\s+dimA: type=int; val=\[(?P<n>\d+),(?P<c>\d+),(?P<h>\d+),(?P<w>\d+)\];')
    # Note: input information is in the 3rd line after xDesc key
    xInfo = xPattern.match(lines[xDescLine + 3])
    if xInfo:
        conv['n'] = int(xInfo.group('n'))
        conv['c'] = int(xInfo.group('c'))
        conv['h'] = int(xInfo.group('h'))
        conv['w'] = int(xInfo.group('w'))
    else:
        print("ERROR: Input dimensions not found at expected spot for block:{}".format(i+1))
        conv['n'] = conv['c'] = conv['h'] = conv['w'] = -1
        
    ## Filter Information
    wDescLine = [x for x in range(len(lines)) if 'wDesc' in lines[x]]
    assert len(wDescLine) == 1, "multiple 'wDesc' in block:{}".format(i+1)
    wDescLine = wDescLine[0]
    
    wPattern = re.compile(r'i!\s+dimA: type=int; val=\[(?P<k>\d+),(?P<i_prime>\d+),(?P<f_h>\d+),(?P<f_w>\d+)\];')
    # NOTE: filter information is in the 4th line after wDesc key.
    wInfo = wPattern.match(lines[wDescLine + 4])
    if wInfo:
#         conv['inDim'] = int(wInfo.group('i_prime')) # this is actually (inDim/groupCount)
        conv['k'] = int(wInfo.group('k'))
        conv['f_h'] = int(wInfo.group('f_h'))
        conv['f_w'] = int(wInfo.group('f_w'))
    else:
        print("ERROR: Filter dimensions not found at expected spot for block:{}.".format(i+1))
        conv['k'] = conv['f_h'] = conv['f_w'] = -1
    
    # Convolution Info
    convDescLine = [x for x in range(len(lines)) if 'convDesc' in lines[x]]
    assert len(convDescLine) == 1, "multiple 'convDesc' in block:{}".format(i+1)
    convDescLine = convDescLine[0]
    
    # Padding
    padPattern = re.compile(r'i!\s+padA: type=int; val=\[(?P<pad_h>\d+),(?P<pad_w>\d+)\];')
    # NOTE: padding information is in the 6th line after convDesc key.
    padInfo = padPattern.match(lines[convDescLine + 6])
    if padInfo:
        conv['pad_h'] = int(padInfo.group('pad_h'))
        conv['pad_w'] = int(padInfo.group('pad_w'))
    else:
        print("ERROR: Padding dimensions not found at expected spot for block:{}.".format(i+1))
        conv['pad_h'] = conv['pad_w'] = -1
    
    # Stride
    stridePattern = re.compile(r'i!\s+strideA: type=int; val=\[(?P<stride_h>\d+),(?P<stride_w>\d+)\];')
    # NOTE: stride info is in the 7th line after convDesc key.
    strideInfo = stridePattern.match(lines[convDescLine + 7])
    if strideInfo:
        conv['stride_h'] = int(strideInfo.group('stride_h'))
        conv['stride_w'] = int(strideInfo.group('stride_w'))
    else:
        print("ERROR: Padding dimensions not found at expected spot for block:{}.".format(i+1))
        conv['stride_h'] = conv['stride_w'] = -1
    
    # Dilation
    dilPattern = re.compile(r'i!\s+dilationA: type=int; val=\[(?P<dil_h>\d+),(?P<dil_w>\d+)\];')
    # NOTE: dilation info is in the 8th line after convDesc key.
    dilInfo = dilPattern.match(lines[convDescLine + 8])
    if dilInfo:
        conv['dil_h'] = int(dilInfo.group('dil_h'))
        conv['dil_w'] = int(dilInfo.group('dil_w'))
    else:
        print("ERROR: Dilation dimensions not found at expected spot for block:{}.".format(i+1))
        conv['dil_h'] = conv['dil_w'] = -1
    
    # Group Count
    groupPattern = re.compile(r'i!\s+groupCount: type=int; val=(?P<groupCount>\d+);')
    # NOTE: groupcount info is in the 9th line after convDesc key.
    groupInfo = groupPattern.match(lines[convDescLine + 9])
    if groupInfo:
        conv['group'] = int(groupInfo.group('groupCount'))
    else:
        print("ERROR: Dilation dimensions not found at expected spot for block:{}.".format(i+1))
        conv['group'] = -1
    
    ## ALGORITHM INFO
    algoLine = [x for x in range(len(lines)) if 'algo' in lines[x]]
    assert len(algoLine) == 1, "multiple 'algo' in block:{}".format(i+1)
    algoLine = algoLine[0]
    algoPattern = re.compile(r'i!\s+algo: type=cudnnConvolutionFwdAlgo_t; val=(?P<algo>[a-zA-Z0-9_ ()]+);')
    algoInfo = algoPattern.match(lines[algoLine])
    if algoInfo:
        conv['fwd_algo'] = algoInfo.group('algo')
    else:
        print("ERROR: Algorithm information not found at expected spot for block:{}.".format(i+1))
        conv['fwd_algo'] = "UNKNOWN"
    # Add to list
    convsInfo.append(conv)

# for item in convsInfo:
#     print(item)
print(len(convsInfo))

104


In [22]:
import pandas as pd
df = pd.DataFrame(convsInfo)
print(len(df))
unique_df = df.drop_duplicates()
print(len(unique_df))
# print(unique_df)

104
24
    n      c    h    w      k  f_h  f_w  pad_h  pad_w  stride_h  stride_w  \
0   4      3  320  320     64    7    7      3      3         2         2   
1   4     64   80   80   1536    1    1      0      0         1         1   
2   4   1536   80   80   1536    3    3      1      1         1         1   
3   4   1536   80   80    256    1    1      0      0         1         1   
4   4     64   80   80    256    1    1      0      0         1         1   
5   4    256   80   80   1536    1    1      0      0         1         1   
11  4    256   80   80   3072    1    1      0      0         1         1   
12  4   3072   80   80   3072    3    3      1      1         2         2   
13  4   3072   40   40    512    1    1      0      0         1         1   
14  4    256   80   80    512    1    1      0      0         2         2   
15  4    512   40   40   3072    1    1      0      0         1         1   
16  4   3072   40   40   3072    3    3      1      1         1      

In [10]:
# print(df)
distinctConv = unique_df.apply(lambda item: "std::make_tuple({},{},{},{},{},{},{},{},{},{},{},{})".format(item['w'], item['h'], item['c'], item['n'],
                                                                           item['k'], item['f_w'], item['f_h'], item['pad_w'],
                                                                           item['pad_h'], item['stride_w'], item['stride_h'],
                                                                           item['group']), axis=1)

In [11]:
# arglist = []
# for item in df:
#     string = "std::make_tuple({},{},{},{},{},{},{},{},{},{},{},{})".format(item['w'], item['h'], item['c'], item['n'],
#                                                                            item['k'], item['f_w'], item['f_h'], item['pad_w'],
#                                                                            item['pad_h'], item['stride_w'], item['stride_h'],
#                                                                            item['groupCount'])
#     arglist.append(string)
 
allconvstr = ",\n".join(distinctConv)
print(allconvstr)

std::make_tuple(320,320,3,4,64,7,7,3,3,2,2,1),
std::make_tuple(80,80,64,4,1536,1,1,0,0,1,1,1),
std::make_tuple(80,80,1536,4,1536,3,3,1,1,1,1,32),
std::make_tuple(80,80,1536,4,256,1,1,0,0,1,1,1),
std::make_tuple(80,80,64,4,256,1,1,0,0,1,1,1),
std::make_tuple(80,80,256,4,1536,1,1,0,0,1,1,1),
std::make_tuple(80,80,256,4,3072,1,1,0,0,1,1,1),
std::make_tuple(80,80,3072,4,3072,3,3,1,1,2,2,32),
std::make_tuple(40,40,3072,4,512,1,1,0,0,1,1,1),
std::make_tuple(80,80,256,4,512,1,1,0,0,2,2,1),
std::make_tuple(40,40,512,4,3072,1,1,0,0,1,1,1),
std::make_tuple(40,40,3072,4,3072,3,3,1,1,1,1,32),
std::make_tuple(40,40,512,4,6144,1,1,0,0,1,1,1),
std::make_tuple(40,40,6144,4,6144,3,3,1,1,2,2,32),
std::make_tuple(20,20,6144,4,1024,1,1,0,0,1,1,1),
std::make_tuple(40,40,512,4,1024,1,1,0,0,2,2,1),
std::make_tuple(20,20,1024,4,6144,1,1,0,0,1,1,1),
std::make_tuple(20,20,6144,4,6144,3,3,1,1,1,1,32),
std::make_tuple(20,20,1024,4,12288,1,1,0,0,1,1,1),
std::make_tuple(20,20,12288,4,12288,3,3,1,1,2,2,32),
std::mak

In [15]:
df.groupby(df.columns.tolist(),as_index=False).size()

In [23]:
# Save dataframe - to be used in parseDeepBench.ipynb
df.to_pickle('/workspace/resnext101_32x48d-archinfo.pkl')